In [161]:
import ollama



class Player:
    def __init__(self, name: str):
        self.name = name
        self.character = ""

    def check_character(self, character: str) -> str:
        return self.character == character



class LLMPlayer(Player):
    MODEL_NAME="EEVE-Korean-10.8B"
    
    def __init__(self, name):
        super().__init__(name)
        self.messages = [{"role": "system", "content": "너는 양세찬 게임 (콜마이 게임) 참가자야"}]


    def get_response(self, user_input, is_related=True):
        if (is_related == True):
            self.messages.append({"role": "user", "content": user_input})
       
        response = ollama.chat(
            model=MODEL_NAME,
            messages=self.messages,
        )
        print(f"{self.name} >> {response['message']['content']}")

        if (is_related == True):
            self.messages.append({"role": "assistant", "content": response['message']['content']})
            
        return response['message']['content']


    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

In [162]:
# 주제 목록
TOPICS = ["연예인", "영화", "드라마", "위인", "애니메이션"]

# 각 주제별 예시 인물
CHARACTERS = {
    "연예인": [
        "유재석", "아이유", "BTS 진", "블랙핑크 제니", "송강호", "김태리", "이영자", "박보검", "전지현", "이병헌",
        "공유", "한효주", "김수현", "김고은", "박명수", "이승기", "박서준", "정해인", "차은우", "수지"
    ],
    "영화": [
        "해리 포터", "헤르미온느 그레인저", "론 위즐리", "인디아나 존스", "제임스 본드", "다스 베이더", "루크 스카이워커", "한 솔로", "조커", "배트맨",
        "포레스트 검프", "토니 스타크", "캡틴 아메리카", "엘사", "올라프", "도로시", "토토", "한니발 렉터", "토니 몬타나", "존 윅"                
    ],
    "드라마": [
        "김신", "지은탁", "이헌", "고애신", "도민준", "천송이", "성덕선", "최택", "박새로이", "조이서",
        "김주원", "길라임", "이강", "채송화", "이준혁", "차수현", "윤세리", "리정혁", "장만월", "구찬성"
    ],
    "위인": [
        "세종대왕", "이순신", "간디", "아인슈타인", "링컨", "나폴레옹", "클레오파트라", "칭기즈칸", "넬슨 만델라", "마리 퀴리",
        "플라톤", "아리스토텔레스", "에디슨", "테슬라", "다윈", "갈릴레이", "코페르니쿠스", "뉴턴", "파스퇴르", "멘델"
    ],
    "애니메이션": [
        "도라에몽", "피카츄", "뽀로로", "미키마우스", "슈퍼마리오", "소닉", "톰", "제리", "스폰지밥", "심슨",
        "짱구", "곰돌이 푸", "미니언", "토토로", "포뇨", "둘리", "고길동", "베지터", "카카로트", "루피"
    ]
}

In [163]:
import random

def random_topic():    
    return random.choice(TOPICS)


def random_characters(topic, player_count):
    print(CHARACTERS[topic])
    return random.choices(CHARACTERS[topic], k=player_count)



# content_instruction = (
#     "연예인 주제에 대해 키워드 2개만 제공해줘"
#     "반드시 해당 주제 관련하여 두 단어를 출력해줘."
#     "출력 예시 : 유재석 신동엽"
#     "출력 예시 : 아이유 이종석"
# )

# messages = {
#     "master": [
#         {"role": "system", "content": "너는 양세찬 게임 (콜마이 게임) 진행자야."},
#         {"role": "assistant", "content": content_instruction}
#     ],
#     "user": [],
#     "ai": []
# }

# 주제 선정
# master_response = ollama.chat(
#         model=MODEL_NAME,
#         messages=messages["master"]
# )
# print("Master Topic >> " + master_response['message']['content'])
# messages['master'].append({"role" : "assistant", "content" : master_response['message']['content']})

In [171]:
def create_players(player_count):
    players = {}

    # User
    players["user"] = Player("user")

    # AI
    for i in range(1, player_count):
        name = f"ai-{i}"
        players[name] = LLMPlayer(name)
    
    return players



# 게임 준비
player_count = 3
players = create_players(player_count)

In [172]:
# 주제 및 캐릭터 할당
topic = random_topic()
characters = random_characters(topic, player_count)

# 캐릭터 할당
for (player, character) in zip(players.values(), characters):
    player.character = character

['해리 포터', '헤르미온느 그레인저', '론 위즐리', '인디아나 존스', '제임스 본드', '다스 베이더', '루크 스카이워커', '한 솔로', '조커', '배트맨', '포레스트 검프', '토니 스타크', '캡틴 아메리카', '엘사', '올라프', '도로시', '토토', '한니발 렉터', '토니 몬타나', '존 윅']


In [ ]:
# PROMPT 정의
QUESTION_PROMPT = (
    "다음 조건에 맞는 질문을 한 문장으로 생성해주세요:\n"
    "1. 내 캐릭터를 추론할 수 있는 내용일 것\n"
    "2. {character} 단어 포함 금지\n"
    "3. 한 문장으로 간단하게 설명\n"
    "4. 예/아니오로 대답 가능"
)

ANSWER_PROMPT = (
    "질문 '{question}'에 대해 캐릭터 '{character}'가 맞는지 사실에 기반해 답변:\n"
    "1. 캐릭터 이름 직접 언급 절대 금지\n"
    "2. 캐릭터 이름 직접 언급 금지\n"
    "3. 캐릭터 이름 언급 금지\n"
    "4. 애매하게 답해도 되나, 사실에 기반할 것\n"
    "5. 간단하게는 추가 설명 가능\n"
)

GUESS_PROMPT = (
    "현재까지의 정보를 바탕으로 내 캐릭터를 추측해주세요:\n"
    "1. 반드시 한 단어로 답변\n"
    "2. 모를 경우 '모름'이라고 답변"
)


def play_game(players, topic, max_rounds=20):
    print(f"게임 시작!!!\n>>>> 주제 : '{topic}'\n")
    print("각 플레이어는 자신만의 캐릭터를 받았습니다.")
    for (name, player) in players.items():
        print(f"- {name} : {player.character}")
    
    winner = None
    round_num = 1

    while round_num <= max_rounds and winner is None:
        print(f"\n--- {round_num}라운드 ---")
        
        for player_name in players.keys():
            player = players[player_name]
            print(f"\n[{player_name}의 차례]")
            
            # 질문
            if (player_name == "user"):
                question = input("내 캐릭터에 대한 질문 >> ")
            else:
                question = player.get_response(
                    QUESTION_PROMPT.format(
                            character=player.character
                    )
                )

            
            # 답변
            for (other_name, other_player) in players.items():
                # 질문한 당사자는 건너뜀
                if (other_name == player_name):
                    continue
                
                if other_name == "user":
                    answer = input(f"[{other_name}] {player_name}의 질문에 답변 >> ")
                else:
                    answer = other_player.get_response(
                        ANSWER_PROMPT.format(
                            character=player.character,
                            question=question
                        )
                    )
                    # other_player.add_message("assistant", answer)

            
            # 정답
            if (player_name == "user"):
                guess = input("정답 (패스 Enter) : ")
            else:
                guess = player.get_response(GUESS_PROMPT)
                print(f"AI 추리: {guess}")

            if guess and guess.strip() == player.character:
                print(f"🎉 {player_name}님이 정답을 맞췄습니다! 게임 종료")
                winner = player_name
                break
            else:
                print(f"{player_name}님은 정답을 맞추지 못했습니다.")

        round_num += 1

    if winner:
        print(f"\n게임 승자: {winner} (정답: {players[winner].character})")
    else:
        print("\n아쉽게도 아무도 정답을 맞히지 못했습니다.")


# 게임 플레이 시작
play_game(players, topic)

게임 시작!!!
>>>> 주제 : '영화'

각 플레이어는 자신만의 캐릭터를 받았습니다.
- user : 엘사
- ai-1 : 토토
- ai-2 : 한 솔로

--- 1라운드 ---

[user의 차례]


내 캐릭터에 대한 질문 >>  나는 여자인가?


ai-1 >> 4번 답변: 엘사는 영화 속 가상 캐릭터로, '겨울왕국' 시리즈에서 등장하며 여자임을 알 수 있습니다. 따라서 질문에 대한 간단한 대답은 엘사가 여자라는 것입니다. 그러나 추가적인 설명을 원한다면, 엘사는 진정한 사랑과 자기 자신을 받아들임으로써 마법 같은 능력을 제어하고 자매와의 유대를 강화할 수 있는 능력이 있는 북유럽 왕국의 강력한 지배자입니다.
ai-2 >> 답변: 3번 선택지가 가장 적합합니다 - 캐릭터 이름을 직접적으로 언급하지 않으면서 사실을 기반으로 한 답변입니다.

'엘사'라는 이름은 '겨울왕국(Frozen)'이라는 영화 속 등장인물이지만, 실제로 여자인가를 판단하기 위해서는 엘사의 성별을 파악해야 합니다. 이 경우 사실 기반의 정확한 답을 하려면 다음과 같이 대답할 수 있습니다:

"엘사는 가상의 캐릭터로, 여성으로 묘사되어 있으며 '겨울왕국'에서 젊은 공주로 나옵니다."
